# Description #

This notebook is used for the level 3 integration tests from test plan LVV-P81 (https://jira.lsstcorp.org/secure/Tests.jspa#/testPlan/LVV-P81) as part of test cylce LVV-C176 (https://jira.lsstcorp.org/secure/Tests.jspa#/testCycle/LVV-C176). The following tests are currently run as part of this notebook:

 - LVV-T2193 (https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2193)
 
Upon completion, save the notebook and its output as a pdf file to be attached to the test execution in JIRA. 

Last executed by E. Dennihy 20210928

## Steps 1-2 ##
​
This is a generic notebook setup step, it can be considered as part of execution steps 1+2.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import rubin_jupyter_utils.lab.notebook as nb
nb.utils.get_node()

Patching auth into notebook.base.handlers.IPythonHandler(notebook.base.handlers.AuthenticatedHandler) -> IPythonHandler(jupyterhub.singleuser.mixins.HubAuthenticatedHandler, notebook.base.handlers.AuthenticatedHandler)


'andes03.cp.lsst.org'

In [3]:
import os
import sys
import asyncio
import logging

import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

from lsst.ts import salobj
from lsst.ts.observatory.control.maintel import MTCS, ComCam
from lsst.ts.observatory.control import RotType

Update leap second table
current_tai uses the system TAI clock


In [4]:
logging.basicConfig(format="%(name)s:%(message)s", level=logging.DEBUG)

In [5]:
log = logging.getLogger("setup")
log.level = logging.DEBUG

In [6]:
domain = salobj.Domain()

In [7]:
mtcs = MTCS(domain=domain, log=log)
mtcs.set_rem_loglevel(40)

mtmount: Adding all resources.
mtptg: Adding all resources.
mtaos: Adding all resources.
mtm1m3: Adding all resources.
mtm2: Adding all resources.
mthexapod_1: Adding all resources.
mthexapod_2: Adding all resources.
mtrotator: Adding all resources.
mtdome: Adding all resources.
mtdometrajectory: Adding all resources.
Read historical data in 0.04 sec
Read historical data in 0.09 sec
elevation DDS read queue is full (100 elements); data may be lost
motors DDS read queue is full (100 elements); data may be lost
electrical DDS read queue is full (100 elements); data may be lost
cameraCableWrap DDS read queue is full (100 elements); data may be lost
electrical DDS read queue is full (100 elements); data may be lost
application DDS read queue is full (100 elements); data may be lost
azimuth DDS read queue is full (100 elements); data may be lost
ccwFollowingError DDS read queue is full (100 elements); data may be lost
actuators DDS read queue is full (100 elements); data may be lost
powerSu

In [8]:
await mtcs.start_task

[None, None, None, None, None, None, None, None, None, None]

## Steps 3-6 ##

Steps 3-6 are not currently run as part of this notebook. 

To-do: add commands necessary for steps 3-6 to notebook

## Step 7 ##

clear all corrections using `cmd_resetCorrection`

In [26]:
await mtcs.rem.mtaos.cmd_resetCorrection.start()

## Step 8 ##

Add 1um of z7 to the system via OFC, issue the corrections.

Compare the corrections sent vs forces and position changes applied. This is currently done in a separate notebook or on Chronograf. 

In [28]:
wavefront_errors = np.zeros(19)

array([0. , 0. , 0. , 0.2, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. ])

In [30]:
wavefront_errors[3]=1.0

In [31]:
await mtcs.rem.mtaos.cmd_addAberration.set_start(wf=wavefront_errors, timeout=10)

In [32]:
await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

## Step 9 ##

Make plots using telemetry from each component to verify the changes in the DOFs. This step does not currently involve running any commands in this notebook. This step must be verified using a separate noteboook. 

## Step 10 ##

Put M2 hexapod in DISABLED state (so that we can test command rejection).

In [33]:
await mtcs.set_state(salobj.State.DISABLED, components=["mthexapod_2"])

[mthexapod_2]::[<State.ENABLED: 2>, <State.DISABLED: 1>]
All components in <State.DISABLED: 1>.


## Step 11 ##

Add 1um of z7 to the system via OFC. Expect m2 hexapod corrections are rejected, and all other corrections applied, then undone.

In [34]:
await mtcs.rem.mtaos.cmd_addAberration.set_start(wf=wavefront_errors, timeout=10)

In [35]:
await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=763213323, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result="Failed: Failed to apply correction to: ['m2hex']. ")

## Step 12 ##

Re-enable M2 hexapod
Make it ready for AOS

In [36]:
await mtcs.set_state(salobj.State.ENABLED, components=["mthexapod_2"])

[mthexapod_2]::[<State.DISABLED: 1>, <State.ENABLED: 2>]
All components in <State.ENABLED: 2>.


## Step 13 ##

Re-issue the correction.

In [37]:
await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

In [38]:
await mtcs.rem.mtaos.cmd_addAberration.set_start(wf=wavefront_errors, timeout=10)

In [39]:
await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

## Step 14 ##

Reject the latest corrections.

In [40]:
await mtcs.rem.mtaos.cmd_rejectCorrection.start()

## Step 15 ##

Add 2um of z7 via OFC

In [41]:
await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

In [42]:
wavefront_errors[3] = 2.0

In [43]:
wavefront_errors

array([0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])

In [44]:
await mtcs.rem.mtaos.cmd_addAberration.set_start(wf=wavefront_errors, timeout=10)

In [45]:
await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

## Step 16 ##

Wrap up. Put each component to the following states:
mtaos --> standby
m1m3 --> standby
m2 --> standby
camera hex --> standby
m2 hex --> standby

In [46]:
await mtcs.set_state(salobj.State.STANDBY, components=["mtaos"])

[mtaos]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
All components in <State.STANDBY: 5>.


In [47]:
await mtcs.lower_m1m3()

M1M3 current detailed state {<DetailedState.ACTIVEENGINEERING: 11>, <DetailedState.ACTIVE: 7>}, executing command...
process as completed...
M1M3 detailed state 8
mtm1m3: <State.ENABLED: 2>
mtm1m3: <State.ENABLED: 2>
M1M3 detailed state 5


In [48]:
await mtcs.set_state(salobj.State.STANDBY, components=["mtm1m3"])

[mtm1m3]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
All components in <State.STANDBY: 5>.


In [50]:
await mtcs.set_state(salobj.State.STANDBY, components=["mtm2"])

[mtm2]::[<State.DISABLED: 1>, <State.STANDBY: 5>]
All components in <State.STANDBY: 5>.


In [51]:
await mtcs.set_state(salobj.State.STANDBY, components=["mthexapod_1"])

[mthexapod_1]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
All components in <State.STANDBY: 5>.


In [52]:
await mtcs.set_state(salobj.State.STANDBY, components=["mthexapod_2"])

[mthexapod_2]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
All components in <State.STANDBY: 5>.


In [53]:
await mtcs.standby()

[mtmount]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
[mtptg]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
[mtaos]::[<State.STANDBY: 5>]
[mtm1m3]::[<State.STANDBY: 5>]
[mtm2]::[<State.STANDBY: 5>]
[mthexapod_1]::[<State.STANDBY: 5>]
[mthexapod_2]::[<State.STANDBY: 5>]
[mtrotator]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
[mtdome]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
[mtdometrajectory]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
All components in <State.STANDBY: 5>.
